In [1]:
import numpy as np
import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import keras.backend as K

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('data/sentipolc_char_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
len(word_index)

127

In [4]:
data = np.load('data/sentipolc_char_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

In [5]:
import pandas as pd
pd.Series(y_train).value_counts()

0    2543
1    1611
dtype: int64

Check the max length of the text

In [6]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(word_index.values())
print('max id {}'.format(max_idx))

max len seq 190
max id 127


In [7]:
len(word_index)

127

In [8]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq, padding='post')
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq, padding='post')

In [9]:
x_train_pad.shape

(4154, 190)

# Define metrics to use

In [10]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
     how many selected items are relevant.
      """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))		
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))		
    precision = true_positives / (predicted_positives + K.epsilon())		
    return precision

In [11]:
def recall(y_true, y_pred):
    """Recall metric.
 
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_test = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_test.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [13]:
callbacks = [
    ReduceLROnPlateau(patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
]

In [22]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx+1, output_dim=50, input_length=max_len_seq))
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu', strides=1))
    model.add(AveragePooling1D())
    model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu', strides=1))
    model.add(GlobalMaxPool1D())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [15]:
model = build_model()
model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 190, 50)           6400      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 190, 128)          32128     
_________________________________________________________________
average_pooling1d_1 (Average (None, 95, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 95, 64)            41024     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3250      
___________________________________________________________

In [16]:
history = model.fit(x_train_pad_split, y_train_pad_split, 
                    validation_data=(x_val_pad_split, y_val_pad_split),
                    batch_size=128, epochs=100, callbacks=callbacks)

Train on 3323 samples, validate on 831 samples
Epoch 1/100
3323/3323 [==============================] - 3s 809us/step - loss: 0.6686 - acc: 0.6097 - precision: 0.0888 - recall: 0.0111 - val_loss: 0.6476 - val_acc: 0.6125 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3323/3323 [==============================] - 0s 75us/step - loss: 0.6389 - acc: 0.6467 - precision: 0.6480 - recall: 0.1881 - val_loss: 0.6219 - val_acc: 0.6871 - val_precision: 0.7418 - val_recall: 0.2943
Epoch 3/100
3323/3323 [==============================] - 0s 75us/step - loss: 0.6263 - acc: 0.6684 - precision: 0.6413 - recall: 0.3371 - val_loss: 0.6108 - val_acc: 0.6955 - val_precision: 0.7514 - val_recall: 0.3190
Epoch 4/100
3323/3323 [==============================] - 0s 75us/step - loss: 0.6147 - acc: 0.6867 - precision: 0.6783 - recall: 0.3752 - val_loss: 0.5932 - val_acc: 0.7004 - val_precision: 0.7359 - val_recall: 0.3571
Epoch 5/100
3323/3323 [==============================] - 0s 75us/step - 

## Evaluate on the test set

In [23]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=12)

Epoch 1/12
4154/4154 [==============================] - 1s 174us/step - loss: 0.6642 - acc: 0.6105
Epoch 2/12
4154/4154 [==============================] - 0s 69us/step - loss: 0.6358 - acc: 0.6553
Epoch 3/12
4154/4154 [==============================] - 0s 69us/step - loss: 0.6167 - acc: 0.6731
Epoch 4/12
4154/4154 [==============================] - 0s 68us/step - loss: 0.6043 - acc: 0.6904
Epoch 5/12
4154/4154 [==============================] - 0s 68us/step - loss: 0.5909 - acc: 0.7022
Epoch 6/12
4154/4154 [==============================] - 0s 69us/step - loss: 0.5735 - acc: 0.7171
Epoch 7/12
4154/4154 [==============================] - 0s 69us/step - loss: 0.5530 - acc: 0.7333
Epoch 8/12
4154/4154 [==============================] - 0s 69us/step - loss: 0.5449 - acc: 0.7364
Epoch 9/12
4154/4154 [==============================] - 0s 69us/step - loss: 0.5276 - acc: 0.7477
Epoch 10/12
4154/4154 [==============================] - 0s 69us/step - loss: 0.5174 - acc: 0.7595
Epoch 11/12
4154/4

In [25]:
loss, acc = model.evaluate(x_test_pad, y_test, 3)

1050/1050 [==============================] - 0s 450us/step


In [27]:
print(loss, acc)

0.6609133741578885 0.6228571532453809


In [19]:
#model.save('models/sentipolc_cnn_char.hdf5')